# Programmet har til formål at få geocoded apoteker og apoteksåbninger. Dette skal bruges til at skabe konkurrence og forbruger parametre. 

In [134]:
import numpy             as np
import matplotlib.pyplot as plt           #We use it for construction of graphs, both 3D and 2D

import pandas as pd

import json
import os
import requests

# spatial stuff
import geopandas as gpd
import fiona
import folium
import shapely
from tqdm import tqdm

In [135]:
import os
current_path = os.getcwd()
current_path

'C:\\Users\\Nicklas Lindegaard\\Desktop\\Speciale\\Python\\thesis_2020'

In [3]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


Importering af Excel oversigt over apoteker

In [124]:
url = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/APOTEK_20200109.csv'
exceldata = pd.read_csv(url , error_bad_lines=False)
exceldata = exceldata.dropna(subset=['Adresse', 'PostNr'])
exceldata


,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region
0,359,Albertslund Apotek,6,Bytorvet 5,2620.0,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0
1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450.0,NaN,NaN,1084.0,201.0,1084.0
2,082-1,Allinge Apotek,7,Kirkegade 13,3770.0,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0
3,175,Allinge Apotek,6,Kirkegade 13,3770.0,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0
4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300.0,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0
...,...,...,...,...,...,...,...,...,...,...
790,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260.0,01Jan1933 0:00:00.000,NaN,1082.0,751.0,1082.0
791,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230.0,01Jan1936 0:00:00.000,NaN,1082.0,751.0,1082.0
792,259,Aars Apotek,6,Søndergade 34,9600.0,01Jan1905 0:00:00.000,NaN,1081.0,820.0,1081.0
793,106,Aarup Apotek,6,Bredgade 25,5560.0,01-01-1859 00:00,31Dec2008 0:00:00.000,1083.0,420.0,1083.0


In [125]:
exceldata_adress = exceldata["Adresse"]
exceldata_postal = exceldata["PostNr"].apply(int)
exceldata["Adresse_v2"] = exceldata["Adresse"] + ', ' + exceldata_postal.astype(str) + ' Denmark'


exceldata_addres_list = list(exceldata["Adresse_v2"])
exceldata

,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Adresse_v2
0,359,Albertslund Apotek,6,Bytorvet 5,2620.0,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0,"Bytorvet 5, 2620 Denmark"
1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450.0,NaN,NaN,1084.0,201.0,1084.0,"Allerød Stationsvej 9, 3450 Denmark"
2,082-1,Allinge Apotek,7,Kirkegade 13,3770.0,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark"
3,175,Allinge Apotek,6,Kirkegade 13,3770.0,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark"
4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300.0,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0,"Strandlodsvej 41, 2300 Denmark"
...,...,...,...,...,...,...,...,...,...,...,...
790,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260.0,01Jan1933 0:00:00.000,NaN,1082.0,751.0,1082.0,"Viby Ringvej 10, ST. 26, 8260 Denmark"
791,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230.0,01Jan1936 0:00:00.000,NaN,1082.0,751.0,1082.0,"Haslevej 17, 8230 Denmark"
792,259,Aars Apotek,6,Søndergade 34,9600.0,01Jan1905 0:00:00.000,NaN,1081.0,820.0,1081.0,"Søndergade 34, 9600 Denmark"
793,106,Aarup Apotek,6,Bredgade 25,5560.0,01-01-1859 00:00,31Dec2008 0:00:00.000,1083.0,420.0,1083.0,"Bredgade 25, 5560 Denmark"


Geocoding


In [51]:
print(type(exceldata_addres_list[0]))

df = gpd.tools.geocode(exceldata_addres_list[700])
df

<class 'str'>


,geometry,address
0,POINT (10.20717 56.15792),"Immervad 2, 8000 Århus C, Denmark"


In [14]:
pip install opencage


Note: you may need to restart the kernel to use updated packages.


In [126]:
from opencage.geocoder import OpenCageGeocode

key = "5ce774970ad843b1bfebabd2ac5c098f"
geocoder = OpenCageGeocode(key)

url = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/APOTEK_20200109.csv'
exceldata = pd.read_csv(url , error_bad_lines=False)
exceldata = exceldata.dropna(subset=['Adresse', 'PostNr'])

exceldata_adress = exceldata["Adresse"]
exceldata_postal = exceldata["PostNr"].apply(int)
exceldata["Adresse_v2"] = exceldata["Adresse"] + ', ' + exceldata_postal.astype(str) + ' Denmark'


exceldata_addres_list = list(exceldata["Adresse_v2"])


In [127]:
latitudes = []
longitudes = []

for address in exceldata_addres_list:
	result = geocoder.geocode(address, no_annotations="1")

	if result and len(result):
		longitude = result[0]['geometry']['lng']
		latitude = result[0]['geometry']['lat']
	else:
		longitude = "N/A"
		latitude = "N/A"

	latitudes.append(latitude)
	longitudes.append(longitude)

In [47]:
from datetime import datetime
type(exceldata["Startdato"][0])

str

,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Adresse_v2,latitudes,longitudes,dato,year
0,359,Albertslund Apotek,6,Bytorvet 5,2620.0,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0,"Bytorvet 5, 2620 Denmark",55.656514,12.353695,1966-01-01,1966.0
2,082-1,Allinge Apotek,7,Kirkegade 13,3770.0,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,2016-05-01,2016.0
3,175,Allinge Apotek,6,Kirkegade 13,3770.0,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,1898-01-01,1898.0
4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300.0,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0,"Strandlodsvej 41, 2300 Denmark",55.664181,12.622773,2015-09-21,2015.0
5,022-1,Antvorskov Apotek,7,Idagårdsvej 1,4200.0,01Jun2018 0:00:00.000,NaN,1085.0,330.0,1085.0,"Idagårdsvej 1, 4200 Denmark",55.390492,11.355228,2018-06-01,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260.0,01Jan1933 0:00:00.000,NaN,1082.0,751.0,1082.0,"Viby Ringvej 10, ST. 26, 8260 Denmark",56.166300,10.198700,1933-01-01,1933.0
791,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230.0,01Jan1936 0:00:00.000,NaN,1082.0,751.0,1082.0,"Haslevej 17, 8230 Denmark",56.156458,10.164773,1936-01-01,1936.0
792,259,Aars Apotek,6,Søndergade 34,9600.0,01Jan1905 0:00:00.000,NaN,1081.0,820.0,1081.0,"Søndergade 34, 9600 Denmark",56.802114,9.519024,1905-01-01,1905.0
793,106,Aarup Apotek,6,Bredgade 25,5560.0,01-01-1859 00:00,31Dec2008 0:00:00.000,1083.0,420.0,1083.0,"Bredgade 25, 5560 Denmark",55.381312,10.049612,1859-01-01,1859.0


In [168]:
exceldata["latitudes"] = latitudes
exceldata["longitudes"] = longitudes

exceldata_startdato_list = list(exceldata["Startdato"])
#exceldata["dato"]      = datetime.fromisoformat(exceldata_startdato_list[:-1])
exceldata["dato"] = pd.to_datetime(exceldata['Startdato'])
exceldata["year"] = pd.DatetimeIndex(exceldata['dato']).year
exceldata.to_excel("Apoteker_Geocoded.xlsx")
exceldata

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Adresse_v2,latitudes,longitudes,dato,year
0,0,0,0,359,Albertslund Apotek,6,Bytorvet 5,2620,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0,"Bytorvet 5, 2620 Denmark",55.656514,12.353695,1966-01-01,1966.0
1,1,1,1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450,NaN,NaN,1084.0,201.0,1084.0,"Allerød Stationsvej 9, 3450 Denmark",55.871124,12.355782,NaT,NaN
2,2,2,2,082-1,Allinge Apotek,7,Kirkegade 13,3770,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,2016-05-01,2016.0
3,3,3,3,175,Allinge Apotek,6,Kirkegade 13,3770,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,1898-01-01,1898.0
4,4,4,4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0,"Strandlodsvej 41, 2300 Denmark",55.664181,12.622773,2015-09-21,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,708,708,790,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260,01Jan1933 0:00:00.000,NaN,1082.0,751.0,1082.0,"Viby Ringvej 10, ST. 26, 8260 Denmark",56.166300,10.198700,1933-01-01,1933.0
709,709,709,791,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230,01Jan1936 0:00:00.000,NaN,1082.0,751.0,1082.0,"Haslevej 17, 8230 Denmark",56.156458,10.164773,1936-01-01,1936.0
710,710,710,792,259,Aars Apotek,6,Søndergade 34,9600,01Jan1905 0:00:00.000,NaN,1081.0,820.0,1081.0,"Søndergade 34, 9600 Denmark",56.802114,9.519024,1905-01-01,1905.0
711,711,711,793,106,Aarup Apotek,6,Bredgade 25,5560,01-01-1859 00:00,31Dec2008 0:00:00.000,1083.0,420.0,1083.0,"Bredgade 25, 5560 Denmark",55.381312,10.049612,1859-01-01,1859.0


In [169]:
exceldata["dummy"] = exceldata["year"].where(exceldata["year"] > 2008, 2008)
#for i in range(2009,2010):
#    exceldata["dummy"] = exceldata["year"].where(exceldata["year"] == i, i)

#exceldata["dummy"] = exceldata["year"].where(exceldata["year"] == 2009, 2009)
exceldata

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Adresse_v2,latitudes,longitudes,dato,year,dummy
0,0,0,0,359,Albertslund Apotek,6,Bytorvet 5,2620,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0,"Bytorvet 5, 2620 Denmark",55.656514,12.353695,1966-01-01,1966.0,2008.0
1,1,1,1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450,NaN,NaN,1084.0,201.0,1084.0,"Allerød Stationsvej 9, 3450 Denmark",55.871124,12.355782,NaT,NaN,2008.0
2,2,2,2,082-1,Allinge Apotek,7,Kirkegade 13,3770,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,2016-05-01,2016.0,2016.0
3,3,3,3,175,Allinge Apotek,6,Kirkegade 13,3770,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,1898-01-01,1898.0,2008.0
4,4,4,4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0,"Strandlodsvej 41, 2300 Denmark",55.664181,12.622773,2015-09-21,2015.0,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,708,708,790,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260,01Jan1933 0:00:00.000,NaN,1082.0,751.0,1082.0,"Viby Ringvej 10, ST. 26, 8260 Denmark",56.166300,10.198700,1933-01-01,1933.0,2008.0
709,709,709,791,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230,01Jan1936 0:00:00.000,NaN,1082.0,751.0,1082.0,"Haslevej 17, 8230 Denmark",56.156458,10.164773,1936-01-01,1936.0,2008.0
710,710,710,792,259,Aars Apotek,6,Søndergade 34,9600,01Jan1905 0:00:00.000,NaN,1081.0,820.0,1081.0,"Søndergade 34, 9600 Denmark",56.802114,9.519024,1905-01-01,1905.0,2008.0
711,711,711,793,106,Aarup Apotek,6,Bredgade 25,5560,01-01-1859 00:00,31Dec2008 0:00:00.000,1083.0,420.0,1083.0,"Bredgade 25, 5560 Denmark",55.381312,10.049612,1859-01-01,1859.0,2008.0


In [132]:
df=exceldata.reset_index(drop=True)
df


,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Adresse_v2,latitudes,longitudes,dato,year,dummy
0,359,Albertslund Apotek,6,Bytorvet 5,2620.0,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0,"Bytorvet 5, 2620 Denmark",55.656514,12.353695,1966-01-01,1966.0,2008.0
1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450.0,NaN,NaN,1084.0,201.0,1084.0,"Allerød Stationsvej 9, 3450 Denmark",55.871124,12.355782,NaT,NaN,2008.0
2,082-1,Allinge Apotek,7,Kirkegade 13,3770.0,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,2016-05-01,2016.0,2016.0
3,175,Allinge Apotek,6,Kirkegade 13,3770.0,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,1898-01-01,1898.0,2008.0
4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300.0,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0,"Strandlodsvej 41, 2300 Denmark",55.664181,12.622773,2015-09-21,2015.0,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260.0,01Jan1933 0:00:00.000,NaN,1082.0,751.0,1082.0,"Viby Ringvej 10, ST. 26, 8260 Denmark",56.166300,10.198700,1933-01-01,1933.0,2008.0
709,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230.0,01Jan1936 0:00:00.000,NaN,1082.0,751.0,1082.0,"Haslevej 17, 8230 Denmark",56.156458,10.164773,1936-01-01,1936.0,2008.0
710,259,Aars Apotek,6,Søndergade 34,9600.0,01Jan1905 0:00:00.000,NaN,1081.0,820.0,1081.0,"Søndergade 34, 9600 Denmark",56.802114,9.519024,1905-01-01,1905.0,2008.0
711,106,Aarup Apotek,6,Bredgade 25,5560.0,01-01-1859 00:00,31Dec2008 0:00:00.000,1083.0,420.0,1083.0,"Bredgade 25, 5560 Denmark",55.381312,10.049612,1859-01-01,1859.0,2008.0


In [170]:
exceldata2 = pd.read_excel(r"C:\Users\Nicklas Lindegaard\Desktop\Speciale\Python\thesis_2020\Apoteker_Geocoded.xlsx")

exceldata
#df_2008 =  exceldata[(exceldata["dummy"]  == '2008.0')]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Adresse_v2,latitudes,longitudes,dato,year,dummy
0,0,0,0,359,Albertslund Apotek,6,Bytorvet 5,2620,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0,"Bytorvet 5, 2620 Denmark",55.656514,12.353695,1966-01-01,1966.0,2008.0
1,1,1,1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450,NaN,NaN,1084.0,201.0,1084.0,"Allerød Stationsvej 9, 3450 Denmark",55.871124,12.355782,NaT,NaN,2008.0
2,2,2,2,082-1,Allinge Apotek,7,Kirkegade 13,3770,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,2016-05-01,2016.0,2016.0
3,3,3,3,175,Allinge Apotek,6,Kirkegade 13,3770,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,1898-01-01,1898.0,2008.0
4,4,4,4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0,"Strandlodsvej 41, 2300 Denmark",55.664181,12.622773,2015-09-21,2015.0,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,708,708,790,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260,01Jan1933 0:00:00.000,NaN,1082.0,751.0,1082.0,"Viby Ringvej 10, ST. 26, 8260 Denmark",56.166300,10.198700,1933-01-01,1933.0,2008.0
709,709,709,791,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230,01Jan1936 0:00:00.000,NaN,1082.0,751.0,1082.0,"Haslevej 17, 8230 Denmark",56.156458,10.164773,1936-01-01,1936.0,2008.0
710,710,710,792,259,Aars Apotek,6,Søndergade 34,9600,01Jan1905 0:00:00.000,NaN,1081.0,820.0,1081.0,"Søndergade 34, 9600 Denmark",56.802114,9.519024,1905-01-01,1905.0,2008.0
711,711,711,793,106,Aarup Apotek,6,Bredgade 25,5560,01-01-1859 00:00,31Dec2008 0:00:00.000,1083.0,420.0,1083.0,"Bredgade 25, 5560 Denmark",55.381312,10.049612,1859-01-01,1859.0,2008.0


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Adresse_v2,latitudes,longitudes,dato,year,dummy
76,76,76,77,115-1,Brande Apotek,7,Storegade 13,7330,01May2009 0:00:00.000,NaN,1082.0,756.0,1082.0,"Storegade 13, 7330 Denmark",55.943996,9.126554,2009-05-01,2009.0,2009.0
83,83,83,85,105-3,Brovst Apotek,7,Østergade 16,9460,01Nov2009 0:00:00.000,NaN,1081.0,849.0,1081.0,"Østergade 16, 9460 Denmark",57.097293,9.524836,2009-11-01,2009.0,2009.0
330,330,330,369,341-2,København Hamlets Apotek,7,Hamletsgade 2,2200,01Jun2009 0:00:00.000,NaN,1084.0,101.0,1084.0,"Hamletsgade 2, 2200 Denmark",55.703088,12.547708,2009-06-01,2009.0,2009.0
433,433,433,500,156-1,Nørre-Snede Apotek,7,Bredgade 22,8766,01Jun2009 0:00:00.000,NaN,1082.0,756.0,1082.0,"Bredgade 22, 8766 Denmark",55.968579,9.397711,2009-06-01,2009.0,2009.0
642,642,642,718,038-1,Videbæk Apotek,7,Bredgade 32,6920,01May2009 0:00:00.000,NaN,1082.0,760.0,1082.0,"Bredgade 32, 6920 Denmark",56.091332,8.630600,2009-05-01,2009.0,2009.0
653,653,653,730,189-A2,Vissenbjerg Apoteksudsalg,8,Østergade 11,5492,01Jan2009 0:00:00.000,18Sep2017 0:00:00.000,1083.0,420.0,1083.0,"Østergade 11, 5492 Denmark",55.385161,10.136660,2009-01-01,2009.0,2009.0
712,712,712,794,189-1,Aarup Apotek,7,Bredgade 60,5560,01Jan2009 0:00:00.000,NaN,1083.0,420.0,1083.0,"Bredgade 60, 5560 Denmark",55.378920,10.047500,2009-01-01,2009.0,2009.0


In [227]:
from tqdm import tqdm
frames_year = {}
for i in range(2008,2021):
    frames_year[str(i)] = exceldata[(exceldata['dummy']  == i)]  # use globals() to store the vars in the global stack
frames_year['2009']


map1 = folium.Map(
    location=[55.656514 , 	12.353695],
    tiles='cartodbpositron',
    zoom_start=12,
)
frames_year['2008'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='blue',).add_to(map1), axis=1)
frames_year['2009'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='red',).add_to(map1), axis=1)
frames_year['2010'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='green',).add_to(map1), axis=1)
frames_year['2011'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='black',).add_to(map1), axis=1)
frames_year['2012'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='cloud',).add_to(map1), axis=1)
frames_year['2013'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='orange',).add_to(map1), axis=1)
frames_year['2014'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='yellow',).add_to(map1), axis=1)
frames_year['2015'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='purple',).add_to(map1), axis=1)
frames_year['2016'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='lightred',).add_to(map1), axis=1)
frames_year['2017'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='lightgreen',).add_to(map1), axis=1)
frames_year['2018'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='gray',).add_to(map1), axis=1)
frames_year['2019'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='pink',).add_to(map1), axis=1)
frames_year['2020'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='lime',).add_to(map1), axis=1)
  
map1

In [226]:
len(frames_year["2016"])+len(frames_year["2017"])+len(frames_year["2018"])+len(frames_year["2019"])

147

In [ ]:
frames_year = {}
for i in range(2008,2021):
    frames_year[str(i)] = exceldata[(exceldata['dummy']  > i)]  # use globals() to store the vars in the global stack
frames_year['2009']

In [221]:
for i in range(1,713):
    if exceldata['dummy'][i] == 2008:
        exceldata['dummy2'][i] = 0
    else:
        exceldata['dummy2'][i] = 1
exceldata

map2 = folium.Map(
    location=[55.656514 , 	12.353695],
    tiles='cartodbpositron',
    zoom_start=12,
)
frames_year['2008'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='blue',).add_to(map1), axis=1)
frames_year['2009'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='red',).add_to(map1), axis=1)
frames_year['2010'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='green',).add_to(map1), axis=1)
frames_year['2011'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='black',).add_to(map1), axis=1)
frames_year['2012'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='cloud',).add_to(map1), axis=1)
frames_year['2013'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='orange',).add_to(map1), axis=1)
frames_year['2014'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='yellow',).add_to(map1), axis=1)
frames_year['2015'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='lightblue',).add_to(map1), axis=1)
frames_year['2016'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='lightred',).add_to(map1), axis=1)
frames_year['2017'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='lightgreen',).add_to(map1), axis=1)
frames_year['2018'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='gray',).add_to(map1), axis=1)
frames_year['2019'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='pink',).add_to(map1), axis=1)
frames_year['2020'].apply(lambda row:folium.CircleMarker(location=[row["latitudes"], row["longitudes"]],radius=1,color='lime',).add_to(map1), axis=1)
  
map1

<ipython-input-221-81a98b3eea52>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exceldata['dummy2'][i] = 0
<ipython-input-221-81a98b3eea52>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exceldata['dummy2'][i] = 1


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,...,Kommune,Region,Adresse_v2,latitudes,longitudes,dato,year,dummy,dummy_2,dummy2
0,0,0,0,359,Albertslund Apotek,6,Bytorvet 5,2620,01Jan1966 0:00:00.000,NaN,...,165.0,1084.0,"Bytorvet 5, 2620 Denmark",55.656514,12.353695,1966-01-01,1966.0,2008.0,0.0,0
1,1,1,1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450,NaN,NaN,...,201.0,1084.0,"Allerød Stationsvej 9, 3450 Denmark",55.871124,12.355782,NaT,NaN,2008.0,0.0,0
2,2,2,2,082-1,Allinge Apotek,7,Kirkegade 13,3770,01May2016 0:00:00.000,NaN,...,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,2016-05-01,2016.0,2016.0,0.0,1
3,3,3,3,175,Allinge Apotek,6,Kirkegade 13,3770,01-01-1898 00:00,01May2016 0:00:00.000,...,400.0,1084.0,"Kirkegade 13, 3770 Denmark",55.277123,14.802304,1898-01-01,1898.0,2008.0,0.0,0
4,4,4,4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300,21Sep2015 0:00:00.000,NaN,...,101.0,1084.0,"Strandlodsvej 41, 2300 Denmark",55.664181,12.622773,2015-09-21,2015.0,2015.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,708,708,790,331,Aarhus Viby Apotek,6,"Viby Ringvej 10, ST. 26",8260,01Jan1933 0:00:00.000,NaN,...,751.0,1082.0,"Viby Ringvej 10, ST. 26, 8260 Denmark",56.166300,10.198700,1933-01-01,1933.0,2008.0,0.0,0
709,709,709,791,337,Aarhus Åbyhøj Apotek,6,Haslevej 17,8230,01Jan1936 0:00:00.000,NaN,...,751.0,1082.0,"Haslevej 17, 8230 Denmark",56.156458,10.164773,1936-01-01,1936.0,2008.0,0.0,0
710,710,710,792,259,Aars Apotek,6,Søndergade 34,9600,01Jan1905 0:00:00.000,NaN,...,820.0,1081.0,"Søndergade 34, 9600 Denmark",56.802114,9.519024,1905-01-01,1905.0,2008.0,0.0,0
711,711,711,793,106,Aarup Apotek,6,Bredgade 25,5560,01-01-1859 00:00,31Dec2008 0:00:00.000,...,420.0,1083.0,"Bredgade 25, 5560 Denmark",55.381312,10.049612,1859-01-01,1859.0,2008.0,0.0,0
